# Optimization problem

m3cUlabs has developed a new vaccine aggainst COVID-19, but it has only been aproved by the Junta de Andalucia. Therefore they can only supply vaccines to each of the 8 provinces of Andalucia. They have done several studies and have developed through inference the porcentage of people that will be inmune once vaccinated given the province and the day of the vaccination campaign. 

This laboratory knows that his vaccine is not the best one developed yet. However they do not want everybody to know it. Therefore they want to carefully supply vaccines to those provinces that will have higher rates of inmunity.

All the same, Junta de Andalucia knows how pharmaceutical operates and their aim to obtain benefits and maintain a good reputation. So they havw impossed some constraints about the supply of the vaccines. There will be needed at least:

- At least X vaccines per province and day
- No more than X vaccines per province and day
- At least X vaccines per province within the whole campaing

Nevertheless m3cUlabs has also their restrictions. They are not able to supply more than X vaccines daily.

The objective of m3cUlabs is to maximize the imunization of the population so they can get a good reputation and expand their bussiness to other parts of Spain.

## Create dataset

As I have created the problem, I also have to create the dataset with the random probabilities of inmunization per province. For that we will create a matrix and use the random funciton provided by numpy. 

In [42]:
import numpy as np
import pandas as pd

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)

Once we have the matrix with the weights we can create a dataframe to show it more clearly. 

In [43]:
import pandas as pd
data=pd.DataFrame(X)
data.rename(index = {0 : 'Malaga', 1:"Sevilla",2:"Cordoba",3:"Cadiz",4:"Huelva",5:"Jaen",6:"Granada",7:"Almeria"}, inplace = True)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Malaga,0.42,0.72,0.00,0.30,0.15,0.09,0.19,0.35,0.40,0.54,0.42,0.69,0.20,0.88,0.03,0.67,0.42,0.56,0.14,0.20
Sevilla,0.80,0.97,0.31,0.69,0.88,0.89,0.09,0.04,0.17,0.88,0.10,0.42,0.96,0.53,0.69,0.32,0.69,0.83,0.02,0.75
Cordoba,0.99,0.75,0.28,0.79,0.10,0.45,0.91,0.29,0.29,0.13,0.02,0.68,0.21,0.27,0.49,0.05,0.57,0.15,0.59,0.70
Cadiz,0.10,0.41,0.69,0.41,0.05,0.54,0.66,0.51,0.94,0.59,0.90,0.14,0.14,0.81,0.40,0.17,0.93,0.35,0.75,0.73
Huelva,0.88,0.62,0.75,0.35,0.27,0.90,0.43,0.96,0.66,0.62,0.11,0.95,0.45,0.58,0.41,0.24,0.90,0.57,0.00,0.62
Jaen,0.33,0.53,0.89,0.36,0.91,0.62,0.02,0.93,0.69,1.00,0.17,0.14,0.93,0.70,0.07,0.76,0.75,0.92,0.71,0.12
Granada,0.02,0.03,0.03,0.25,0.86,0.54,0.55,0.84,0.12,0.28,0.59,0.97,0.56,0.02,0.80,0.23,0.81,0.39,0.86,0.75
Almeria,0.56,0.14,0.06,0.12,0.04,0.11,0.23,0.71,0.56,0.01,0.07,0.97,0.57,0.20,0.25,0.74,0.20,0.58,0.97,0.85


## Formulate mathematically the problem as a linear optimization model (based on the general formulation of an LP). Identify the model sets, parameters, variables, objective function and constraints.